In [10]:
!pip install tensorflow


In [12]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#Load preprocessed data 
file_path = 'data_output/processed_data.csv'
data = pd.read_csv(file_path)

#Seperate features and labels
x = data.drop(colums=['label']).values

#Normalize Data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

#Split the data into training and test sets (only normal data for traing)
x_train, x_test = train_test_split(x_scaled, test_size=0.2, random_state=42)

#Define autoencoder model
input_dim = x_train.shape[1]
encoding_dim = 32 #Size of encoded representation

input_layer = keras.Input(shape=(input_dim))
encoded = layers.Dense(encoding_dim, avtivation='relu')(input_layer)
encoded = layers.Dense(16, activation='relu')(encoded)
decoded = layers.Dense(encoding_dim, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_layers, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

#Train the autoencoder
history = autoencoder.fit(x_train, x_train, epochs=50, batch_size=32, validation_date=(x_test, x_test))

#Plot training loss
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#Save model
autoencoder.save('models/autoencoder.h5')

#Use autoencoder to detect anomalies 
reconstructed = autoencoder.predict(x_scaled)
mse = np.mean(np.power(x_scaled - reconstructed, 2), axis=1)
threshold = np.percentile(mse, 95) #Define threshold for anomaly detection
anomalies =  mse > threshold

#Save anomaly detection result
data['anomaly'] = anomalies
data.to_csv('output_data/anomalu_results.csv', index=False)

#print anomaly summary
print("Anomaly detection results saved. Number of anomalies detected: ", np.sum(anomalies))